<a href="https://colab.research.google.com/github/seinfinity/Leetcode/blob/main/1_ReActor_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Gourieff/ReActor-UI

이 코드는 GitHub에서 ReActor-UI repository를 clone하여 개인 데이터를 활용한 faceswap을 수행합니다. 아래는 각 단계에 대한 간략한 설명입니다. Fast and Simple Face Swap

1. required packages 설치
2. ReActor-UI repository clone
3. required library 설치
4. google drive mount
5. GFPGAN model file 복사
6. source image(아이유), target video(아나운서) 데이터 경로 설정
7. faceswap 실행
8. faceswap 결과물 video 다운로드

#### GPU 연결

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 13 04:32:51 2024       \n+---------------------------------------------------------------------------------------+\n| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |\n|-----------------------------------------+----------------------+----------------------+\n| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |\n| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |\n|                                         |                      |               MIG M. |\n|=========================================+======================+======================|\n|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |\n| N/A   46C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |\n|                                         |                      |                  N/A |\n+-----------------------------------------+--------------

#### 1. required packages 설치

In [2]:
!pip install torch torchvision torchaudio
!pip install -U gdown
!apt-get install -y libgl1-mesa-glx

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 96.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 88.9 MB/s eta 0:

#### 2. ReActor-UI repository clone

In [3]:
# Clone the ReActor-UI repository
!git clone https://github.com/Gourieff/ReActor-UI.git

fatal: destination path 'ReActor-UI' already exists and is not an empty directory.


In [4]:
# Change directory to ReActor-UI
%cd ReActor-UI

/content/ReActor-UI


#### 3. required library 설치

In [5]:
# Install the required python packages
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring torch: markers 'sys_platform == "darwin"' don't match your environment
Ignoring torchvision: markers 'sys_platform == "darwin"' don't match your environment
Ignoring onnxruntime: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring tensorflow: markers 'sys_platform == "darwin"' don't match your environment


#### 4. google drive mount

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 5. GFPGAN model file 복사 / 6. source image(아이유), target video(아나운서) 데이터 경로 설정

In [11]:
import os
!cp /content/drive/MyDrive/HIT_Selena/faceswap/data/models/GFPGANv1.4.pth

source_image_path = '/content/drive/MyDrive/HIT_Selena/faceswap/data/IU.png'  # 업로드한 아이유 사진 경로
target_video_path = '/content/drive/MyDrive/HIT_Selena/faceswap/data/C2654_fps25_2min.mp4'  # 업로드한 아나운서 비디오 경로
output_path = '/content/drive/MyDrive/HIT_Selena/faceswap/data/1_output.mp4'
trimmed_video_path = '/content/drive/MyDrive/HIT_Selena/faceswap/data/C2654_fps25_2min_trimmed.mp4'


cp: missing destination file operand after '/content/drive/MyDrive/HIT_Selena/faceswap/data/models/GFPGANv1.4.pth'
Try 'cp --help' for more information.


In [13]:
!ffmpeg -i {target_video_path} -ss 0 -t 00:02:00 -c copy {trimmed_video_path}
print(trimmed_video_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#### 7. faceswap 실행

In [15]:
!python run.py -s {source_image_path} -t {trimmed_video_path} -o {output_path} --execution-provider cuda --keep-fps --keep-audio

# !python run.py -s {source_image_path} -t {target_video_path} -o {output_path} --execution-provider cuda --keep-fps --keep-audio

y
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
100% 2752/

#### 8. faceswap 결과물 video 다운로드

In [16]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>